In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from scipy.signal import find_peaks
from scipy import stats
from sklearn import svm
from tensorflow import keras
import keras
#from keras import layers
CUDA_LAUNCH_BLOCKING = 1

from tensorflow import keras
from tensorflow.keras import layers

import warnings
warnings.filterwarnings('ignore')


In [2]:
def seed_everything (seed, tf2_set_seed = True, torch_set_seed = True):
    import random, os
    import numpy as np

    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

    if torch_set_seed == True:
        import torch
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

    if tf2_set_seed == True:
        import tensorflow as tf
        tf.random.set_seed(seed)


In [3]:
seed_everything(1, True, False)

In [4]:
path_bvp_train = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/train/bvp.npy'
path_eda_train = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/train/eda.npy'
path_eeg_train = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/train/eeg.npy'
path_personality_train = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/train/personality.npy'
path_temp_train = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/train/temp.npy'


path_bvp_val = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/val/bvp.npy'
path_eda_val = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/val/eda.npy'
path_eeg_val = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/val/eeg.npy'
path_personality_val = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/val/personality.npy'
path_temp_val = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/val/personality.npy'


path_bvp_test = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/test/bvp.npy'
path_eda_test = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/test/eda.npy'
path_eeg_test = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/test/eeg.npy'
path_personality_test = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/test/personality.npy'
path_temp_test = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/test/temp.npy'


path_label_train = '/home/nghia/PR_LAB/KERC_Chall/kerc2021/KERC21Dataset/KERC21Dataset/kerc21/train/label.npy'

bvp_train = np.load(path_bvp_train)
eda_train = np.load(path_eda_train)
eeg_train = np.load(path_eeg_train)
personality_train = np.load(path_personality_train)
temp_train = np.load(path_temp_train)

bvp_test = np.load(path_bvp_test)
eda_test = np.load(path_eda_test)
eeg_test = np.load(path_eeg_test)
personality_test = np.load(path_personality_test)
temp_test = np.load(path_temp_test)

bvp_val = np.load(path_bvp_val)
eda_val = np.load(path_eda_val)
eeg_val = np.load(path_eeg_val)
personality_val = np.load(path_personality_val)
temp_val = np.load(path_temp_val)


label_train = np.load(path_label_train)


In [5]:
bvp_train = (bvp_train - np.mean(bvp_train))/np.std(bvp_train)
eda_train = (eda_train - np.mean(eda_train))/np.std(eda_train)
eeg_train = (eeg_train - np.mean(eeg_train))/np.std(eeg_train)
personality_train = (personality_train - np.mean(personality_train))/np.std(personality_train)
temp_train = (temp_train - np.mean(temp_train))/np.std(temp_train)

bvp_val = (bvp_val - np.mean(bvp_val))/np.std(bvp_val)
eda_val = (eda_val - np.mean(eda_val))/np.std(eda_val)
eeg_val = (eeg_val - np.mean(eeg_val))/np.std(eeg_val)
personality_val = (personality_val - np.mean(personality_val))/np.std(personality_val)
temp_val = (temp_val - np.mean(temp_val))/np.std(temp_val)

bvp_test = (bvp_test - np.mean(bvp_test))/np.std(bvp_test)
eda_test = (eda_test - np.mean(eda_test))/np.std(eda_test)
eeg_test = (eeg_test - np.mean(eeg_test))/np.std(eeg_test)
personality_test = (personality_test - np.mean(personality_test))/np.std(personality_test)
temp_test = (temp_test - np.mean(temp_test))/np.std(temp_test)


In [6]:
bvp_train = np.expand_dims(bvp_train, axis=-1)
eda_train = np.expand_dims(eda_train, axis=-1)
personality_train = np.expand_dims(personality_train, axis=-1)
temp_train = np.expand_dims(temp_train, axis=-1)


bvp_val = np.expand_dims(bvp_val, axis=-1)
eda_val = np.expand_dims(eda_val, axis=-1)
personality_val = np.expand_dims(personality_val, axis=-1)
temp_val = np.expand_dims(temp_val, axis=-1)


bvp_test = np.expand_dims(bvp_test, axis=-1)
eda_test = np.expand_dims(eda_test, axis=-1)
personality_test = np.expand_dims(personality_test, axis=-1)
temp_test = np.expand_dims(temp_test, axis=-1)


In [7]:
bvp_val.shape, eda_val.shape, eeg_val.shape, personality_val.shape, temp_val.shape


((234, 3841, 1), (234, 241, 1), (234, 15360, 14), (234, 5, 1), (234, 5, 1))

In [8]:
dct_label = {'HAHV': 0, 'HALV': 1, 'LAHV': 2, 'LALV': 3}
new_label = []
for x in label_train:
    new_label.append(dct_label[x])
new_label = np.asarray(new_label)

In [9]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


def build_model(
    inputs,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    return layers.Dense(64, activation="sigmoid")(x)
    # outputs = layers.Dense(n_classes, activation="sigmoid")(x)
    # return keras.Model(inputs, outputs)

input_bvp_shape = bvp_train.shape[1:]
input_eda_shape = eda_train.shape[1:]
input_eeg_shape = eeg_train.shape[1:]
input_personality_shape = personality_train.shape[1:]
input_temp_shape = temp_train.shape[1:]


input_bvp = keras.Input(shape=input_bvp_shape)
input_eda = keras.Input(shape=input_eda_shape)
input_eeg = keras.Input(shape=input_eeg_shape)
input_personality = keras.Input(shape=input_personality_shape)
input_temp = keras.Input(shape=input_temp_shape)


model_bvp = build_model(
    input_bvp,
    head_size=64,
    num_heads=8,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
model_eda = build_model(
    input_eda,
    head_size=64,
    num_heads=8,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model_eeg = build_model(
    input_eeg,
    head_size=64,
    num_heads=8,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model_personality = build_model(
    input_personality,
    head_size=64,
    num_heads=8,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model_temp = build_model(
    input_temp,
    head_size=64,
    num_heads=8,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

combined = layers.concatenate([model_bvp, model_eda, model_eeg, model_personality, model_temp])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = layers.Dense(256, activation="relu")(combined)
z = layers.Dense(4, activation="sigmoid")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model = keras.Model([input_bvp, input_eda, input_eeg, input_personality, input_temp], z)


2021-10-30 11:54:04.256799: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-30 11:54:04.287960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-30 11:54:04.288773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-30 11:54:04.289695: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [10]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 3841, 1)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 241, 1)]     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 15360, 14)]  0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 5, 1)]       0                                            
______________________________________________________________________________________________

In [11]:
# config = tf.ConfigProto(device_count = {'GPU': 0})
# sess = tf.Session(config=config)
# keras.backend.set_session(sess)

# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_trans.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
##binary_crossentropy_sparse_categorical_crossentropy
model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=2e-4),
    metrics=["accuracy"],
)
# model.summary()

# callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    [bvp_train, eda_train, eeg_train, personality_train, temp_train],
    new_label,
    validation_split=0.15,
    epochs=100,
    batch_size=200,
    callbacks=callbacks,
)

# model.evaluate([x_gsr_test, x_hr_test, x_rr_test, x_temp_test], y_gsr_test, verbose=1)

2021-10-30 11:54:06.872002: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2021-10-30 11:54:18.175396: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2021-10-30 11:54:24.198153: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2021-10-30 11:54:25.602945: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 16.01MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-10-30 11:54:25.604244: W tensorflow/core/kernels/gpu_utils.cc:49] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2021-10-30 11:54:25.624444: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or dir

ResourceExhaustedError:  OOM when allocating tensor with shape[48200,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/multi_head_attention_4/value/einsum/Einsum (defined at tmp/ipykernel_5320/2243652583.py:33) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_27603]

Function call stack:
train_function
